In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [2]:
import numpy as np
import torch
import scanpy as sc
import muon as mu
import anndata
import mudata
import pandas as pd

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
DATA_DIR = "/media/yob/nacho_stuff/prostate_cancer/Integration/MVAE-PoE/OtF-prostate/data/"
mdata = mu.read(DATA_DIR + "mdata_preprocessed_final_filtered.h5mu")
mdata

MuData object with n_obs × n_vars = 42475 × 31460
  obs:	'sample', 'tissue', 'patient', 'ann', 'imagerow', 'imagecol'
  2 modalities
    rna:	42475 x 18950
      obs:	'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'size_factors', 'sample', 'type', 'patient', 'ann', 'tissue', 'imagerow', 'imagecol'
      obsm:	'spatial'
      layers:	'counts', 'scran_normalization', 'spotclean'
    msi:	42475 x 12510
      obs:	'ST_barcode', 'sample', 'tissue', 'patient', 'ann', 'imagerow', 'imagecol'
      obsm:	'spatial'

In [5]:
from src.model import MVAE, MVAEParams

mvae_params = MVAEParams(beta=0.01, n_layers=2, z_dim=100, n_hidden=300)

In [6]:
torch.set_num_threads(16)
model = MVAE(mdata, mvae_params, use_cuda=True)
model

N batches for mod1:  16
N batches for mod2:  16
(42475, 18950)
(42475, 12510)


MVAE(
  (rna): ModalityLayers(
    (shared_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
    )
    (batch_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=1600, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=1600, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
    )
    (private_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.

In [7]:
from src.train import train_mvae, TrainParams
train_params = TrainParams(train_size=0.8, n_epochs=100, learning_rate=1e-4, batch_size=128)

In [15]:
torch.autograd.set_detect_anomaly(True)
hist = train_mvae(model, mdata, train_params)

Train data size: 33980
Test data size: 8495


  0%|                                                   | 0/266 [00:00<?, ?it/s]/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/site-packages/torch/autograd/__init__.py:173: UserWarning: Error detected in MmBackward0. Traceback of forward call that caused the error:
  File "/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/media/gambino/students_workdir/anna/miniconda3/envs/ak/

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [4, 128]], which is output 0 of AsStridedBackward0, is at version 1040; expected version 1020 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [61]:
unique_batches = torch.tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15]])

neighbor_batches = torch.tensor([[ 2],
        [ 7],
        [15],
        [ 0],
        [ 6],
        [10],
        [ 2],
        [ 4],
        [ 0],
        [ 6],
        [ 0],
        [ 9],
        [ 5],
        [14],
        [ 3],
        [ 1],
        [15],
        [ 8],
        [ 5],
        [ 5],
        [ 1],
        [ 4],
        [ 0],
        [ 7],
        [ 8],
        [15],
        [ 7],
        [15],
        [11],
        [14],
        [ 5],
        [14],
        [ 6],
        [11],
        [ 8],
        [ 9],
        [ 4],
        [12],
        [ 9],
        [ 1],
        [14],
        [11],
        [ 3],
        [14],
        [13],
        [ 1],
        [ 1],
        [12],
        [ 5],
        [ 9],
        [11],
        [ 2],
        [ 0],
        [10],
        [12],
        [ 8],
        [ 9],
        [11],
        [14],
        [12],
        [ 8],
        [ 4],
        [ 9],
        [11],
        [ 4],
        [12],
        [14],
        [ 7],
        [ 3],
        [10],
        [ 3],
        [13],
        [ 6],
        [ 7],
        [14],
        [11],
        [ 6],
        [ 5],
        [ 9],
        [ 4],
        [12],
        [13],
        [ 8],
        [10],
        [ 3],
        [10],
        [ 4],
        [ 1],
        [ 4],
        [ 6]]).long()

P = torch.tensor([0.1278, 0.1076, 0.0999, 0.0691, 0.0485, 0.0427, 0.0419, 0.0328, 0.0328,
        0.0321, 0.0317, 0.0255, 0.0226, 0.0191, 0.0175, 0.0159, 0.0157, 0.0125,
        0.0121, 0.0116, 0.0114, 0.0100, 0.0094, 0.0089, 0.0072, 0.0068, 0.0067,
        0.0058, 0.0053, 0.0051, 0.0050, 0.0047, 0.0044, 0.0043, 0.0042, 0.0038,
        0.0036, 0.0034, 0.0032, 0.0032, 0.0031, 0.0027, 0.0025, 0.0024, 0.0023,
        0.0022, 0.0022, 0.0021, 0.0021, 0.0020, 0.0020, 0.0018, 0.0018, 0.0017,
        0.0016, 0.0015, 0.0015, 0.0014, 0.0014, 0.0014, 0.0014, 0.0013, 0.0012,
        0.0012, 0.0011, 0.0011, 0.0010, 0.0010, 0.0010, 0.0010, 0.0009, 0.0009,
        0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0007, 0.0007,
        0.0007, 0.0007, 0.0006, 0.0005, 0.0005, 0.0004, 0.0004, 0.0004, 0.0004])
P.shape, neighbor_batches.shape

(torch.Size([90]), torch.Size([90, 1]))

In [66]:
target = torch.zeros_like(unique_batches.squeeze(), dtype=torch.float)
target

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [67]:
target.scatter_add_(0, neighbor_batches.squeeze(), P)

RuntimeError: Index tensor must have the same number of dimensions as self tensor

In [65]:
from src.harmony import kmeans

In [33]:
clusters, initial = kmeans(X, 5)

In [34]:
clusters

tensor([0, 2, 0, 2, 0, 2, 3, 0, 4, 3, 2, 2, 3, 4, 0, 1, 0, 1, 1, 4])

In [35]:
initial

tensor([[0.5152, 0.6251, 0.4648, 0.2677, 0.8065],
        [0.9214, 0.7754, 0.6769, 0.6754, 0.6930],
        [0.4996, 0.1874, 0.4587, 0.4212, 0.2909],
        [0.3072, 0.7661, 0.3963, 0.7026, 0.4590],
        [0.2935, 0.9202, 0.7524, 0.4808, 0.1952]])